In [1]:
print("OK")

OK


In [2]:
%pwd

'c:\\Users\\hp\\Desktop\\PROJECT\\Medical_AI_Chatbot\\research'

In [3]:
import os
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\hp\\Desktop\\PROJECT\\Medical_AI_Chatbot'

In [5]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [6]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents

In [7]:
extracted_data = load_pdf_file(data=r"C:\Users\hp\Desktop\PROJECT\Medical_AI_Chatbot\Data")


In [8]:
from typing import List
from langchain.schema import Document

def filter_to_minimal_docs(docs: List[Document]) -> List[Document]:
    """
    Given a list of Document objects, return a new list of Document objects
    containing only 'source' in metadata and the original page_content.
    """
    minimal_docs: List[Document] = []
    for doc in docs:
        src = doc.metadata.get("source")
        minimal_docs.append(
            Document(
                page_content=doc.page_content,
                metadata={"source": src}
            )
        )
    return minimal_docs

In [9]:
minimal_docs = filter_to_minimal_docs(extracted_data)

In [10]:
#Split the Data into Text Chunks
def text_split(minimal_docs):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(minimal_docs)
    return text_chunks

In [11]:
text_chunks=text_split(minimal_docs)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 5859


In [12]:
from langchain.embeddings import HuggingFaceEmbeddings

In [13]:
#Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

embeddings = download_hugging_face_embeddings()

C:\Users\hp\AppData\Local\Temp\ipykernel_12320\220312833.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
c:\Users\hp\anaconda3\envs\medibot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
W0715 22:45:22.183000 12320 site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


In [14]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [15]:
from dotenv import load_dotenv
import os

load_dotenv()

# Now fetch the value
together_api_key = os.getenv("TOGETHER_API_KEY")

# Debug print to verify
print("✅ Together Key:", together_api_key[:8] if together_api_key else "❌ Not Found")


✅ Together Key: ❌ Not Found


In [16]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
TOGETHER_API_KEY=os.environ.get('TOGETHER_API_KEY')

In [20]:
from dotenv import load_dotenv
import os

# Load from .env
load_dotenv()

# Get the keys
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
TOGETHER_API_KEY = os.getenv("TOGETHER_API_KEY")




In [25]:
import os
from dotenv import load_dotenv

load_dotenv()  # ✅ Load values from .env file

# Get the keys
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
TOGETHER_API_KEY = os.getenv("TOGETHER_API_KEY")

# Safely set environment variables only if they exist
if isinstance(PINECONE_API_KEY, str) and isinstance(TOGETHER_API_KEY, str):
    os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
    os.environ["TOGETHER_API_KEY"] = TOGETHER_API_KEY
    print("✅ API keys loaded into environment successfully!")
else:
    raise ValueError("❌ API keys not found or invalid. Check your `.env` file.")


✅ API keys loaded into environment successfully!


In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()

print("Pinecone Key:", os.getenv("PINECONE_API_KEY")[:10])
print("Environment:", os.getenv("PINECONE_ENVIRONMENT"))


✅ Pinecone Key: pcsk_3YiyZ
✅ Environment: us-east-1


In [27]:
import os
from dotenv import load_dotenv

load_dotenv()
print("ENV:", repr(os.getenv("PINECONE_ENVIRONMENT")))


ENV: 'us-east-1'


In [28]:
from dotenv import load_dotenv
import os
from pinecone import Pinecone

# Load .env variables
load_dotenv()

# Get Pinecone API key securely
api_key = os.getenv("PINECONE_API_KEY")

# Initialize Pinecone with key
pc = Pinecone(api_key=api_key)

# Check existing indexes (safe usage)
print(pc.list_indexes().names())


['medical-bot']


In [29]:
from pinecone import ServerlessSpec

index_name = "medical-bot"  # change if desired

if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )

index = pc.Index(index_name)

In [30]:
# 1. Load PDFs
from langchain.document_loaders import PyPDFLoader, DirectoryLoader

def load_pdf_file(data):
    loader = DirectoryLoader(data, glob="*.pdf", loader_cls=PyPDFLoader)
    return loader.load()

extracted_data = load_pdf_file(r"C:\Users\hp\Desktop\PROJECT\Medical_AI_Chatbot\Data")



In [31]:
# 2. Filter metadata
from typing import List
from langchain.schema import Document

def filter_to_minimal_docs(docs: List[Document]) -> List[Document]:
    return [
        Document(page_content=doc.page_content, metadata={"source": doc.metadata.get("source")})
        for doc in docs
    ]

minimal_docs = filter_to_minimal_docs(extracted_data)


In [32]:
# 3. Split into chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter

def text_split(minimal_docs):
    splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    return splitter.split_documents(minimal_docs)

text_chunks = text_split(minimal_docs)


In [33]:
# 4. Initialize Embeddings
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


In [34]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings,
)


In [35]:
# Load Existing index 

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

## Add more data to existing database

In [36]:
dswith = Document(
    page_content="dswithbappy is a youtube channel that provides tutorials on various topics.",
    metadata={"source": "Youtube"}
)

In [37]:
docsearch.add_documents(documents=[dswith])

['0755110d-d5d2-4008-90de-6d4db9219630']

In [38]:
docsearch

In [39]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [40]:
retrieved_docs = retriever.invoke("What is Acne?")
retrieved_docs

[Document(id='d0fe1478-9e93-4ddd-82ca-c364c5cea8e7', metadata={'source': 'C:\\Users\\hp\\Desktop\\PROJECT\\Medical_AI_Chatbot\\Data\\Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='a5921869-84de-4189-9ae1-2d7f15e927fd', metadata={'source': 'C:\\Users\\hp\\Desktop\\PROJECT\\Medical_AI_Chatbot\\Data\\Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='31c8f6de-40ce-4f0f-b74c-b7f400505ae3', metadata={'source': 'C:\\Users\\hp\\Desktop\\PROJECT\\Medical_AI_Chatbot\\Data\\Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed. (Photograph by Biophoto Associ-\nates, Photo Researchers, Inc. Reproduced by permission.)\nGEM - 0001 to 0432 - A  10/22/03 1:41 P

In [ ]:
from langchain_together import ChatTogether
import os
from dotenv import load_dotenv

load_dotenv()

chatModel = ChatTogether(
    together_api_key=os.getenv("TOGETHER_API_KEY"),
    model="meta-llama/Llama-3-8b-chat-hf", 
)


In [43]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [45]:
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [46]:
question_answer_chain = create_stuff_documents_chain(chatModel, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [47]:
response = rag_chain.invoke({"input": "what is Antibiotic-associated colitis?"})
print(response["answer"])

Antibiotic-associated colitis is a condition that occurs when antibiotics disrupt the balance of gut bacteria, leading to inflammation and damage in the colon. It can cause symptoms such as nausea, vomiting, diarrhea, and fever, and can potentially lead to complications like dehydration, electrolyte imbalances, and perforation of the colon.


In [48]:
response = rag_chain.invoke({"input": "what is Acne?"})
print(response["answer"])

Acne is a skin disorder in which the sebaceous glands become inflamed. It is characterized by the formation of comedones (blackheads and whiteheads), papules, pustules, nodules, and cysts on the skin.
